In [1]:
#import libraries

import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as soup

In [2]:
# Set URL as a variable
url = 'https://nces.ed.gov/programs/digest/d97/d97t078.asp'

# Use requests.post becuase it'll pull data after the webpage has loaded
r=requests.post(url)

# create a "soup" object 
data = soup(r.text, 'html.parser')

In [3]:
# Identify element tag with target data
data.select('table td pre')[0]

<pre style="text-align:left; padding-left:4px">__________________________________________________________________________________________________________________________________________________________________________    
                      |                                                                               |                                                        | Percent
                      |                    Current dollars                                            |                 Constant 1996-97 dollars\1\            | change,
                      |                                                                               |                                                        |1979-80 to
        State         |_______________________________________________________________________________|________________________________________________________|1996-97 in
                      |       |        |        |            |            |             |         

In [4]:
# Our data of interest is the text that is formatted like a table
# Select that data and return as text
doi= data.select('table td pre')[0].text
print(doi)

__________________________________________________________________________________________________________________________________________________________________________    
                      |                                                                               |                                                        | Percent
                      |                    Current dollars                                            |                 Constant 1996-97 dollars\1\            | change,
                      |                                                                               |                                                        |1979-80 to
        State         |_______________________________________________________________________________|________________________________________________________|1996-97 in
                      |       |        |        |            |            |             |             |        |        |        |         |     

In [5]:
# read_csv was taking in object as a file name
# trouble shooting said to use io.StringIO
#  

import io

salary_df = pd.read_csv(io.StringIO(doi),sep='|', #separate columns by |-symbol
            skiprows=6,
            usecols=list(range(0,8))
            )
salary_df

,,1969-70,1979-80,1989-90,1993-94,1994-95,1995-96,1996-97
0,______________________,_______,________,________,____________,____________,_____________,_____________
1,1,2,3,4,5,6,7,8
2,______________________,_______,________,________,____________,____________,_____________,_____________
3,United States .....,"$8,626","$15,970","$31,367","\2\ $35,733","$36,609","\2\ $37,560","\2\ $38,509"
4,,_______,________,________,____________,____________,_____________,_____________
...,...,...,...,...,...,...,...,...
61,Washington ...........,"9,225","18,820","30,457","35,863","36,151","37,853","37,860"
62,West Virginia.........,"7,650","13,710","22,842","30,549","31,944","32,155","33,159"
63,Wisconsin ............,"8,963","16,006","31,921","35,990","37,746","38,182","\2\ 38,950"
64,Wyoming ..............,"8,232","16,012","28,141","30,952","31,285","31,571","31,721"


### Clean the df

In [6]:
# Rename first column
salary_df.rename(columns={salary_df.columns[0]:'State'}, inplace=True)

salary_df.columns

Index(['State', '1969-70', '1979-80 ', '1989-90 ', '    1993-94 ',
       '    1994-95 ', '     1995-96 ', '     1996-97 '],
      dtype='object')

In [7]:
# Remove white space from other column names

# Create an empty dicitonary
new_column_names = {}

# Loop thru column names and remove white space
for i in list(range(len(salary_df.columns))):
    new_column_names[salary_df.columns[i]] = salary_df.columns[i].strip()

salary_df.rename(columns=new_column_names, inplace=True)

salary_df.head()

,State,1969-70,1979-80,1989-90,1993-94,1994-95,1995-96,1996-97
0,______________________,_______,________,________,____________,____________,_____________,_____________
1,1,2,3,4,5,6,7,8
2,______________________,_______,________,________,____________,____________,_____________,_____________
3,United States .....,"$8,626","$15,970","$31,367","\2\ $35,733","$36,609","\2\ $37,560","\2\ $38,509"
4,,_______,________,________,____________,____________,_____________,_____________


In [8]:
# remove '...' from state name column and then set as index
def period_remover(value):
    
    temp = value.strip()
    
    if len(temp) > 1 and temp[0].lower() in ('abcdefghijklmnopqrstuvwxyz'):
        return temp.split('.')[0].strip()
    else:
        return np.NaN

salary_df_clean = salary_df.copy()

salary_df_clean['State'] = salary_df_clean['State'].apply(period_remover)

salary_df_clean

,State,1969-70,1979-80,1989-90,1993-94,1994-95,1995-96,1996-97
0,NaN,_______,________,________,____________,____________,_____________,_____________
1,NaN,2,3,4,5,6,7,8
2,NaN,_______,________,________,____________,____________,_____________,_____________
3,United States,"$8,626","$15,970","$31,367","\2\ $35,733","$36,609","\2\ $37,560","\2\ $38,509"
4,NaN,_______,________,________,____________,____________,_____________,_____________
...,...,...,...,...,...,...,...,...
61,Washington,"9,225","18,820","30,457","35,863","36,151","37,853","37,860"
62,West Virginia,"7,650","13,710","22,842","30,549","31,944","32,155","33,159"
63,Wisconsin,"8,963","16,006","31,921","35,990","37,746","38,182","\2\ 38,950"
64,Wyoming,"8,232","16,012","28,141","30,952","31,285","31,571","31,721"


In [9]:
cleaner_salary_df = salary_df_clean.dropna(axis=0)
cleaner_salary_df.head(10)

,State,1969-70,1979-80,1989-90,1993-94,1994-95,1995-96,1996-97
3,United States,"$8,626","$15,970","$31,367","\2\ $35,733","$36,609","\2\ $37,560","\2\ $38,509"
5,Alabama,"6,818","13,060","24,828","28,705","31,144","31,313","32,549"
6,Alaska,"10,560","27,210","43,153","47,512","47,951","\2\ 49,620","\2\ 50,647"
7,Arizona,"8,711","15,054","29,402","31,800","32,175","\2\ 32,484","\2\ 33,350"
8,Arkansas,"6,307","12,299","22,352","28,098","28,934","29,322","\2\ 29,975"
9,California,"10,315","18,020","37,998","40,264","41,078","42,259","\2\ 43,474"
11,Colorado,"7,761","16,205","30,758","33,826","34,571","35,364","\2\ 36,175"
12,Connecticut,"9,262","16,229","40,461","49,769","50,045","50,254","50,426"
13,Delaware,"9,015","16,148","33,377","37,469","39,076","40,533","41,436"
14,District of Columbia,"10,285","22,190","38,402","42,543","43,700","43,700","\2\ 45,012"


In [10]:
def data_to_int(val):
    return int(''.join(re.findall(r'\d',val.split('\\')[-1])))

In [11]:
data_to_int('11,001')

11001

In [12]:
salary_df_final = cleaner_salary_df.copy()

for i in list(range(len(salary_df_final.columns)-1)):
    salary_df_final[salary_df_final.columns[i+1]] = salary_df_final[salary_df_final.columns[i+1]].apply(data_to_int)

In [13]:
salary_df_final

,State,1969-70,1979-80,1989-90,1993-94,1994-95,1995-96,1996-97
3,United States,8626,15970,31367,35733,36609,37560,38509
5,Alabama,6818,13060,24828,28705,31144,31313,32549
6,Alaska,10560,27210,43153,47512,47951,49620,50647
7,Arizona,8711,15054,29402,31800,32175,32484,33350
8,Arkansas,6307,12299,22352,28098,28934,29322,29975
9,California,10315,18020,37998,40264,41078,42259,43474
11,Colorado,7761,16205,30758,33826,34571,35364,36175
12,Connecticut,9262,16229,40461,49769,50045,50254,50426
13,Delaware,9015,16148,33377,37469,39076,40533,41436
14,District of Columbia,10285,22190,38402,42543,43700,43700,45012


In [14]:
salary_df_final.to_csv('Data/1997_avg_teacher_salaries.csv',index=False)